# Image captioning

## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from models import Encoder, Decoder

from pathlib import Path
from get_loader import MSCOCODataset, get_loader

from data_prep_utils import *
from utils import train, save_model, load_model, plot_loss
import json
import numpy as np
import time

###### download the data we need
# !cd ~/INM706-image-captioning/Datasets/coco/images/
# !wget http://images.cocodataset.org/zips/train2017.zip
# !wget http://images.cocodataset.org/zips/val2017.zip
# !unzip train2017.zip
# !unzip val2017.zip
# !rm train2017.zip
# !rm val2017.zip

##### run code below if nltk hasn't been set up in clound instance yet
# !python -m nltk.downloader -d /usr/local/share/nltk_data all

###### run code below to save pre-trained weights if needed
# cd ~/INM706-image-captioning/model
# !wget https://download.pytorch.org/models/resnet152-394f9c45.pth
# !mv resnet152-394f9c45.pth resnet152_model.pth

### Choose hyper parameters

In [18]:
# for building vocab
FREQ_THRESHOLD = 5

# for data loader
BATCH_SIZE = 32
CAPS_PER_IMAGE = 5 # how many captions for each image to include in data set

# for encoder and decoder
EMBED_SIZE = 512 # dimension of vocab embedding vector
HIDDEN_SIZE = 512
NUM_LAYERS = 1 #hidden layers in LTSM

# optimiser parameters
OPT_PARAMS = {'lr':1e-3, 'weight_decay':1e-3, 'betas':(0.9, 0.999), 'eps':1e-08}

# training parameters
TOTAL_EPOCH = 15
CHECKPOINT = '../model/image_captioning_model_v9.pth'
PRINT_EVERY = 300 # run print_every batches and then
# print running results. For bigger batch size make this 
# number smaller if you want to see regular output

## Load dataset and dataloader

In [19]:
root = Path('../Datasets/coco')
imgs_path = root/'images'/'train2017'
imgs_path_test = root/'images'/'val2017'

# prepare_datasets(train_percent = 0.87, super_categories=['sports'],
#                     max_train=15000, max_val=3000, max_test=3000)

train_captions_path = root/'annotations'/'sports_captions_train.json'
val_captions_path = root/'annotations'/'sports_captions_val.json'
test_captions_path = root/'annotations'/'sports_captions_test.json'

#### build vocab using full original coco train
# build_vocab(freq_threshold=FREQ_THRESHOLD)

# load vocab
with open('../vocabulary/string_to_index.json') as json_file:
    word2idx = json.load(json_file)

In [20]:
# to boost the performence of CUDA use:
# torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
train_loader_params = {
    'images_path': imgs_path,
    'captions_path': train_captions_path,
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': CAPS_PER_IMAGE,
    'mode': "train",
    'transform': None,
    'batch_size': BATCH_SIZE,
    'shuffle': True,
    'word2idx': word2idx,
}

val_loader_params = {
    'images_path': imgs_path,
    'captions_path': val_captions_path,
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': 1,
    'mode': "train",
    'transform': None,
    'batch_size': BATCH_SIZE,
    'shuffle': True,
    'word2idx': word2idx,
}

train_dl, train_dataset = get_loader(**train_loader_params)
val_dl, val_dataset = get_loader(**val_loader_params)

vocab_size = len(train_dataset.word2idx)

### Encoder and decoder

In [22]:
encoder = Encoder(embed_size=EMBED_SIZE, pretrained=True, model_weight_path="../model/resnet152_model.pth")
decoder = Decoder(embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, vocab_size=vocab_size, num_layers=NUM_LAYERS)

### Training paramaters

In [23]:
# the loss is a cross entropy loss and ignore the index of <PAD> since it doesn't make any difference
# not sure we need this with pad method in pytorch
criterion = nn.CrossEntropyLoss(ignore_index=train_dataset.word2idx["<PAD>"]) ############

# combine the parameters of decoder and encoder
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# Adam optimizer
optimizer = optim.Adam(params, **OPT_PARAMS)

#### Load checkpoints if they exist

In [24]:
if Path(CHECKPOINT).exists():
    encoder, decoder, training_loss, validation_loss = load_model(encoder, decoder, CHECKPOINT)
else:
    print(f'{CHECKPOINT} file does not exist, training startging from scratch')
    training_loss = None
    validation_loss = None

## Train the model

In [ ]:
train_params = {
    'device': device,
    'encoder': encoder,
    'decoder': decoder,
    'criterion': criterion,
    'optimizer': optimizer,
    'train_loader': train_dl,
    'val_loader': val_dl,
    'total_epoch': TOTAL_EPOCH,
    'training_loss': training_loss,
    'validation_loss': validation_loss,
    'checkpoint_path': CHECKPOINT,
    'print_every': PRINT_EVERY
}

training_loss, validation_loss = train(**train_params) 

Step: [0/2344] || Average Training Loss: 2.9743
Step: [300/2344] || Average Training Loss: 2.7166
Step: [600/2344] || Average Training Loss: 2.6994
Step: [900/2344] || Average Training Loss: 2.6813
Step: [1200/2344] || Average Training Loss: 2.6757
Step: [1500/2344] || Average Training Loss: 2.6719
Step: [1800/2344] || Average Training Loss: 2.6694
Step: [2100/2344] || Average Training Loss: 2.6685
Step: [0/94] || Average Validation Loss: 2.6069
######################################################################
Epoch: [0/15] || Training Loss = 2.66 || Training Perplexity: 14.36 || Validation Loss: 2.49 || Validation Perplexity: 12.04|| Time: 12.039838
######################################################################
Step: [0/2344] || Average Training Loss: 2.3694
Step: [300/2344] || Average Training Loss: 2.6512
Step: [600/2344] || Average Training Loss: 2.6415
Step: [900/2344] || Average Training Loss: 2.6379
Step: [1200/2344] || Average Training Loss: 2.6430
Step: [1500/2344